# SWMT on shelf for PanAntarctic 1/20th

This needing intake-ifying due to an issue with `cc.getvar` function.

In [2]:
import intake
import glob

import cartopy.crs as ccrs
import cf_xarray as cfxr
import cmocean as cm
import dask.distributed
import gsw
import matplotlib.pyplot as plt
from matplotlib import gridspec 
import matplotlib.colors as mcolors
import matplotlib.path as mpath
import numpy as np
import xarray as xr
import pdb
import os
import pathlib
import shutil

import warnings # ignore these warnings
warnings.filterwarnings("ignore", category = FutureWarning)
warnings.filterwarnings("ignore", category = UserWarning)
warnings.filterwarnings("ignore", category = RuntimeWarning)

In [3]:
#Starting a dask client
from os import environ
environ["PYTHONWARNINGS"] = "ignore"

from dask.distributed import Client
client = Client(threads_per_worker = 1)

In [3]:
def get_variables(expt, freq, start_time, end_time, lon_slice, lat_slice, model):

    # The models require different diagnostics to calculate the heat and salt fluxes.
    # mom6 outputs a net flux, whilst with mom5 we need to work with the separate components.
    model_vars = {
                    "mom5": {
                        "temperature":           ["temp"],
                        "salinity":              ["salt"],
                        "water_flux_into_ocean": ["pme_river"],
                        "salt_flux":             ["sfc_salt_flux_ice", "sfc_salt_flux_restore"],
                        "heat_flux":             ["sfc_hflux_coupler", "sfc_hflux_from_runoff",
                                                  "frazil_3d_int_z", "sfc_hflux_pme"],
                        "area":                  ["area_t"],
                        "maximum_depth":         ["ht"],
                    },
                    "mom6": {
                        "temperature":           ["thetao"],
                        "salinity":              ["so"],
                        "water_flux_into_ocean": ["wfo"],
                        "salt_flux":             ["salt_flux"],
                        "heat_flux":             ["hfds"],
                        "area":                  ["areacello"],
                        "maximum_depth":         ["deptho"],
                    }
                }
    # Load variables in a dictionary
    ds = {}
    keys = list(model_vars[model].keys())
    for k in keys:
        ds[k] = {}
        for var in model_vars[model][k]:
            if k in ["area", "maximum_depth"]:
                ds[k][var] = cc.querying.getvar(expt, var, session, n = 1) 
                ds[k][var] = ds[k][var].cf.sel(X = lon_slice, Y = lat_slice)
            else:
                try:
                    ds[k][var] = cc.querying.getvar(expt, var, session,
                                                frequency = freq,
                                                start_time = start_time,
                                                end_time = end_time,
                                                chunks = {'time': 1})
                    ds[k][var] = ds[k][var].cf.sel(X = lon_slice, Y = lat_slice, time = slice(start_time, end_time))
                
                    # Correct temperatures (if in K convert to C)
                    if k == 'temperature' and np.max(ds[k][var]) > 100:
                        ds[k][var] = ds[k][var] - 273.15
    
                    # If 3D field, grab the surface
                    if ds[k][var].cf.axes.get('Z'):
                        surface_z = ds[k][var].cf['Z'][0].values
                        ds[k][var] = ds[k][var].cf.sel(Z = 0, method = 'nearest')
                except Exception as e:
                    raise Exception(f"Data loading error: var={var}, freq={freq}, start_time={start_time}, end_time={end_time}, expt={expt}: {str(e)}")

    # Get temperature and salinity to calculate few other things we'll need later on
    SP = ds['salinity'][model_vars[model]['salinity'][0]]
    CT = ds['temperature'][model_vars[model]['temperature'][0]]

    # Calculate pressure
    pressure = gsw.p_from_z(-surface_z, SP.cf['Y']).rename('pressure')

    # Calculate absolute salinity
    SA = gsw.SA_from_SP(SP, pressure, SP.cf['X'], SP.cf['Y']).rename('SA')

    # Ensure we have conservative temperature; Convert MOM6's potential temperature to conservative
    if model == 'mom6':
        CT = gsw.CT_from_pt(SA, CT)
        ds['temperature'][model_vars[model]['temperature'][0]].data = CT.values

    # Calculate potential density
    pot_rho_0 = gsw.sigma0(SA, CT)#.rename('pot_rho_11')

    # Save everything to our dictionary
    ds['pressure'] = pressure
    ds['SA'] = SA
    ds['pot_rho_0'] = pot_rho_0
    
    # Calculate days per month accounting for leap years
    months_standard_noleap = np.array([31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31])
    months_standard_leap = np.array([31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31])
    if 'ryf' or 'panan' in expt:
        nyears = len(np.unique(CT['time.year']))
        days_per_month = np.tile(months_standard_noleap, nyears)
    elif 'iaf' in expt:
        nyears = len(np.unique(CT['time.year']))
        if CT['time.year'][0] % 4 == 0:
            days_per_month = months_standard_leap
        else: 
            days_per_month = months_standard_noleap
        for yr in CT['time.year'][::12][1:]:
            if yr % 4 == 0:
                days_per_month = np.concatenate([days_per_month, months_standard_leap])
            else:
                days_per_month = np.concatenate([days_per_month, months_standard_noleap])
    days_per_month = xr.DataArray(days_per_month, dims = ['time'], coords = {'time': CT['time']}, name = 'days_per_month')
    ds['days_per_month'] = days_per_month

    return ds

In [4]:
def compute_salt_transformation(ds):
    
    # First retrieve temperature and water_flux as an xarray instead of a dictionary
    CT = xr.Dataset(ds['temperature']).to_array().squeeze().drop_vars('variable')

    # Multiply the water flux by absolute salinity to get it in the correct units
    water_flux_into_ocean = xr.Dataset(ds['water_flux_into_ocean']).to_array().squeeze().drop_vars('variable')
    water_flux_into_ocean = ds['SA'] * water_flux_into_ocean

    # Caculate the haline contraction coefficient
    haline_contraction = gsw.beta(ds['SA'], CT, ds['pressure']).rename('beta')

    # Calculate the net salt flux and multiply by 1000 to convert units
    net_salt_flux = xr.Dataset(ds['salt_flux']).to_array().sum(dim = 'variable') * 1000

    # Note that we also multiply pme_river by absolute salinity to have the correct units
    salt_transformation = haline_contraction * (water_flux_into_ocean - net_salt_flux) * ds['days_per_month']
    salt_transformation = salt_transformation.load()

    return salt_transformation

def compute_heat_transformation(ds):

    # First retrieve temperature as an xarray instead of a dictionary
    CT = xr.Dataset(ds['temperature']).to_array().squeeze().drop_vars('variable')

    # Calculate the thermal expansion coefficient 
    thermal_expansion = gsw.alpha(ds['SA'], CT, ds['pressure']).rename('alpha')
    
    # Calculate the net surface heating
    net_surface_heating = xr.Dataset(ds['heat_flux']).to_array().sum(dim = 'variable')
    
    # Calculate the heat transformation
    heat_transformation = thermal_expansion * net_surface_heating * ds['days_per_month']
    heat_transformation = heat_transformation.load()

    return heat_transformation

In [5]:
def isopycnal_bins(ds, salt_transformation, heat_transformation):

    # Next section does a few things. It cycles through isopycnal bins, determines which cells are 
    # within the given bin for each month, finds the transformation values for those cells for each month, 
    # and sums these through time. You are left with an array of shape (isopyncal bins * lats * lons) 
    # where the array associated with a given isopycnal bin is NaN everywhere except where pot_rho_0
    # was within the bin, there it has a time summed transformation value.
    
    # Choose appropriate bin range
    isopycnal_bins = np.arange(31, 33.5, 0.02)  # 125 bins - 31, 33.5, 0.02 (sigma1)
    #isopycnal_bins = np.concatenate([np.arange(25.0, 26.5, 0.05), np.arange(26.5, 28.5, 0.02)])  # 130 bins (sigma0)
    bin_bottoms = isopycnal_bins[:-1]
    isopycnal_bin_mid = (isopycnal_bins[1:] + bin_bottoms) / 2
    isopycnal_bin_diff = np.diff(isopycnal_bins)

    pot_rho_0 = ds['pot_rho_0']

    results_salt = []
    results_heat = []

    for i in range(len(bin_bottoms)):
        # Create binary mask for each bin
        bin_mask = xr.where((pot_rho_0 > bin_bottoms[i]) & (pot_rho_0 <= isopycnal_bins[i + 1]), 1, np.nan)

        # Multiply and sum over time
        salt_sum = (salt_transformation * bin_mask).sum(dim='time')
        heat_sum = (heat_transformation * bin_mask).sum(dim='time')

        results_salt.append(salt_sum.expand_dims({'isopycnal_bins': [isopycnal_bin_mid[i]]}))
        results_heat.append(heat_sum.expand_dims({'isopycnal_bins': [isopycnal_bin_mid[i]]}))

    # Concatenate results along isopycnal dimension
    salt_transformation = xr.concat(results_salt, dim='isopycnal_bins')
    heat_transformation = xr.concat(results_heat, dim='isopycnal_bins')

    # Normalise by number of days and bin thickness
    ndays = ds['days_per_month'].sum()
    c_p = 3992.1 # J kg-1 degC-1

    salt_transformation /= ndays
    heat_transformation /= (c_p * ndays)

    salt_transformation /= isopycnal_bin_diff[:, np.newaxis, np.newaxis]
    heat_transformation /= isopycnal_bin_diff[:, np.newaxis, np.newaxis]

    # Overwrite zeros with NANs 
    # (Note: the code within the for-loop should provide nans but lazy computing with dask can sometimes give unpredictable results)
    salt_transformation = salt_transformation.where(salt_transformation != 0)
    heat_transformation = heat_transformation.where(heat_transformation != 0)

    # Change the sign so that positive means conversion into denser water masses
    salt_transformation *= -1
    heat_transformation *= -1

    # Renaming
    salt_transformation.name = "salt_transformation"
    heat_transformation.name = "heat_transformation"

    return salt_transformation.load(), heat_transformation.load()

## Opening the saved data

In [ ]:
# Importing the saved files
salt_files_C = np.sort(glob.glob('/g/data/g40/kc5856/access_panan_ssp126_SWMT/old_Pan005_SWMT_shelf/binned_salt_trans_pan005_C_*'))
heat_files_C = np.sort(glob.glob('/g/data/g40/kc5856/access_panan_ssp126_SWMT/old_Pan005_SWMT_shelf/binned_heat_trans_pan005_C_*'))

salt_files_M = np.sort(glob.glob('/g/data/g40/kc5856/access_panan_ssp126_SWMT/old_Pan005_SWMT_shelf/binned_salt_trans_pan005_M_*'))
heat_files_M = np.sort(glob.glob('/g/data/g40/kc5856/access_panan_ssp126_SWMT/old_Pan005_SWMT_shelf/binned_heat_trans_pan005_M_*'))

In [ ]:
salt_tr_C = xr.open_mfdataset(salt_files_C, concat_dim='time', combine='nested', chunks={'time': 1, 'yh': 100, 'xh': 100})
heat_tr_C = xr.open_mfdataset(heat_files_C, concat_dim='time', combine='nested', chunks={'time': 1, 'yh': 100, 'xh': 100})
salt_tr_C = salt_tr_C.to_array().squeeze()
heat_tr_C = heat_tr_C.to_array().squeeze()

In [ ]:
salt_tr_C

In [ ]:
salt_tr_M = xr.open_mfdataset(salt_files_M, concat_dim='time', combine='nested', chunks={'time': 1, 'yh': 100, 'xh': 100})
heat_tr_M = xr.open_mfdataset(heat_files_M, concat_dim='time', combine='nested', chunks={'time': 1, 'yh': 100, 'xh': 100})
salt_tr_M = salt_tr_M.to_array().squeeze()
heat_tr_M = heat_tr_M.to_array().squeeze()

In [ ]:
net_tr_C = salt_tr_C + heat_tr_C
net_tr_M = salt_tr_M + heat_tr_M

In [ ]:
# Defining the data separately for variables like area, depth, etc.
expt_con = 'panant-005-zstar-ACCESSyr2'
db = '/home/272/kc5856/databases/honours_test.db'
session = cc.database.create_session(db)
freq = '1 monthly'

lon_slice = slice(None, None)
lat_slice = slice(None, -59)

start_time = '2001-01-01'
end_time = '2001-12-31'

ds_cont = get_variables(expt_con, freq, start_time, end_time, lon_slice, lat_slice, model = 'mom6')

#### Antarctic shelf dense water formation

In [ ]:
def shelf_mask_isobath(var, model_dict):

    paths = {
             "mom5": "/g/data/ik11/grids/Antarctic_slope_contour_1000m.npz",
             "mom6_01": "/g/data/ik11/grids/Antarctic_slope_contour_1000m_MOM6_01deg.nc",
            "mom6_005": "/g/data/ik11/grids/Antarctic_slope_contour_1000m_MOM6_005deg.nc"
             }

    var = var.cf.sel({'latitude': slice(-90, -59)})

    if paths[model_dict][-3:] == '.nc':
        shelf_mask = xr.open_dataset(paths[model_dict])['contour_masked_above']
    else:
        contour_file = np.load(paths[model_dict])
        shelf_mask = xr.DataArray(contour_file['contour_masked_above'],
                                  coords = var.coords, 
                                  dims = var.dims,
                                  name = 'contour_masked_above')
    
    shelf_mask = xr.where(shelf_mask == 0, 1, 0)
    masked_var = var * shelf_mask
    
    return masked_var, shelf_mask

In [ ]:
depth = xr.Dataset(ds_cont['maximum_depth']).to_array().squeeze()
land_mask = (0 * depth).fillna(1)
depth_shelf, shelf_mask = shelf_mask_isobath(depth, 'mom6_01')

In [ ]:
area = xr.Dataset(ds_cont['area']).to_array().squeeze().drop_vars('variable')
varr = xr.Dataset(ds_cont['salinity']).to_array().squeeze().isel(time=0)
area_corr = (area*(varr*0+1)).drop_vars('variable')

In [ ]:
area_corr = area_corr.drop_vars(['time','z_l'])

In [ ]:
# Check mask
area_corr.plot()

Corrected area not needed because it is already being multiplied with a variable with the correct mask below?

In [ ]:
swmt_shelf_C = (net_tr_C * area_corr / 1e6).where(shelf_mask == 1)
heat_shelf_C = (heat_tr_C * area_corr / 1e6).where(shelf_mask == 1)
salt_shelf_C = (salt_tr_C * area_corr / 1e6).where(shelf_mask == 1)

swmt_shelf_C = swmt_shelf_C.chunk({'time': 1, 'yh': 100, 'xh': 100})
heat_shelf_C = heat_shelf_C.chunk({'time': 1, 'yh': 100, 'xh': 100})
salt_shelf_C = salt_shelf_C.chunk({'time': 1, 'yh': 100, 'xh': 100})

In [ ]:
#swmt_shelf_C.cf.sum(['xh', 'yh']).astype('float32') # if needed

In [ ]:
swmt_shelf_sum_C = swmt_shelf_C.cf.sum(['xh', 'yh']).compute()

In [ ]:
# Time mean
swmt_shelf_sum_ave_C = swmt_shelf_sum_C.mean(dim='time')

for da in [swmt_shelf_sum_ave_C]:#, heat_shelf_sum_ave_C, salt_shelf_sum_ave_C]:
    da.attrs["units"] = "Sv"

In [ ]:
swmt_shelf_M = (net_tr_M * area_corr / 1e6).where(shelf_mask == 1)
heat_shelf_M = (heat_tr_M * area_corr / 1e6).where(shelf_mask == 1)
salt_shelf_M = (salt_tr_M * area_corr / 1e6).where(shelf_mask == 1)

swmt_shelf_M = swmt_shelf_M.chunk({'time': 1, 'yh': 100, 'xh': 100})
heat_shelf_M = heat_shelf_M.chunk({'time': 1, 'yh': 100, 'xh': 100})
salt_shelf_M = salt_shelf_M.chunk({'time': 1, 'yh': 100, 'xh': 100})

In [ ]:
swmt_shelf_sum_M = swmt_shelf_M.cf.sum(['xh', 'yh']).compute()

In [ ]:
# Time mean
swmt_shelf_sum_ave_M = swmt_shelf_sum_M.mean(dim='time')

for da in [swmt_shelf_sum_ave_M]: #, heat_shelf_sum_ave_M, salt_shelf_sum_ave_M]:
    da.attrs["units"] = "Sv"

In [ ]:
swmt_shelf_sum_ave_C # check

In [ ]:
swmt_shelf_sum_ave_C.compute()

In [ ]:
swmt_shelf_sum_ave_M.compute()

In [ ]:
# 25% and 75% thresholds
## Replace when using new data
cont_d25 = 32.65
cont_d75 = 32.51

In [ ]:
fig, ax1 = plt.subplots(figsize=(4.5, 5.33))

ax1.plot(swmt_shelf_sum_ave_C, swmt_shelf_sum_ave_C['isopycnal_bins'], color = 'k', label='Control: 3.70 Sv')
ax1.plot(swmt_shelf_sum_ave_M, swmt_shelf_sum_ave_M['isopycnal_bins'], color = 'r', label='Meltwater: 3.11 Sv')
ax1.plot([0, 0], [31, 33.2], 'k', linewidth=0.5)
ax1.plot([-5, 15], [cont_d25, cont_d25], 'k--', linewidth=1)
ax1.plot([-5, 15], [cont_d75, cont_d75], 'k--', linewidth=1)
ax1.set_ylim((33.2, 31))
ax1.set_xlim((-1.5, 10))
ax1.yaxis.set_label_position("right")
ax1.yaxis.tick_right()
ax1.set_ylabel('Density $\sigma_1$ (kg m$^{-3}$)')
ax1.set_xlabel('Surface water-mass transformation (Sv)')
ax1.legend();

In [ ]:
swmt_shelf_sum_ave_C.to_netcdf('/g/data/g40/kc5856/access_panan_ssp126_SWMT/old_Pan005_SWMT_shelf/plots/swmt_shelf_sum_ave_C_pan005.nc')
swmt_shelf_sum_ave_M.to_netcdf('/g/data/g40/kc5856/access_panan_ssp126_SWMT/old_Pan005_SWMT_shelf/plots/swmt_shelf_sum_ave_M_pan005.nc')

#### SWMT map

In [ ]:
transformation_density = 32.58 ## Replace when using new data
shelf_subduction_plot_C = net_tr_C.sel(isopycnal_bins = transformation_density, method = 'nearest') * 1e5
swmt_xt_C = depth.cf['X']
swmt_yt_C = depth.cf['Y']

shelf_subduction_plot_M = net_tr_M.sel(isopycnal_bins = transformation_density, method = 'nearest') * 1e5
swmt_xt_M = depth.cf['X']
swmt_yt_M = depth.cf['Y']

In [ ]:
shelf_subduction_plot_C.astype('float32').compute()

In [ ]:
shelf_subduction_plot_M.astype('float32').compute()

In [ ]:
anom_xt = swmt_xt_M.fillna(0) - swmt_xt_C.fillna(0)
anom_yt = swmt_yt_M.fillna(0) - swmt_yt_C.fillna(0)
shelf_subduction_plot_anom = shelf_subduction_plot_M.fillna(0) - shelf_subduction_plot_C.fillna(0)

In [ ]:
# Taking time means
shelf_subduction_plot_ave_anom = shelf_subduction_plot_anom.mean(dim='time')
shelf_subduction_plot_ave_anom

In [ ]:
shelf_subduction_plot_ave_anom = shelf_subduction_plot_ave_anom.load()

In [ ]:
# Check before saving
shelf_subduction_plot_ave_anom.plot(vmin=-2,vmax=2, cmap=cm.cm.balance)

In [ ]:
shelf_subduction_plot_ave_anom.to_netcdf('/g/data/g40/kc5856/access_panan_ssp126_SWMT/old_Pan005_SWMT_shelf/plots/swmt_shelf_plot_anom_pan005.nc')

#### Finding the 25% and 75%

Control max = 8.05866571 Sv (at density 32.41).  
- 25% of max (2.0146664275 Sv) = 32.65
- 75% of max (6.0439992825 Sv) = 32.51
- Control total transport in this slice = 25.8736142 Sv
- Mean of control in this slice = 3.6962306 Sv.
      
Melt max = 7.71676875 Sv (at density 32.41).  
- 25% of max (1.9291921875 Sv) = BLANK
- 75% of max (5.7875765625) = BLANK
- Melt total transport in this slice = 21.77198215 Sv.
- Mean of meltwater in this region = 3.11028317 Sv.

We just use the control lines as our formation region. 

In [ ]:
# Finding maximum
swmt_shelf_sum_ave_C.sel(isopycnal_bins=slice(32.55,32.69)) # Pick slice based on plot, test for max value

In [ ]:
max_value = 
swmt_shelf_sum_ave_C.isel(isopycnal_bins=max_value).load()

Finding the sums: (change density values below to match 25 and 75 thresholds)

In [ ]:
val_25 = 32.65
val_75 = 32.51

In [ ]:
swmt_shelf_sum_ave_C.sel(isopycnal_bins=slice(val_75,val_25)).compute().mean()

In [ ]:
swmt_shelf_sum_ave_M.sel(isopycnal_bins=slice(val_75,val_25)).compute().mean()

#### Time series

In [ ]:
swmt_shelf_time_C = swmt_shelf_sum_C.sel(isopycnal_bins=slice(val_75,val_25)).mean('isopycnal_bins').compute()

In [ ]:
swmt_shelf_time_M = swmt_shelf_sum_M.sel(isopycnal_bins=slice(val_75,val_25)).mean('isopycnal_bins').compute()

In [ ]:
swmt_shelf_time_C.plot(label="Control")
swmt_shelf_time_M.plot(label="Meltwater")
plt.ylabel("Surface water mass transformation (Sv)")
plt.xlabel("Year")
plt.legend(loc="lower left")
plt.title("Panan01 Surface Water Mass Transformation over 25-75% range")

In [ ]:
# Saving
swmt_shelf_time_C.to_netcdf('/g/data/g40/kc5856/access_panan_ssp126_SWMT/SWMT_timeseries/swmt_shelf_time_pan005_C.nc')
swmt_shelf_time_M.to_netcdf('/g/data/g40/kc5856/access_panan_ssp126_SWMT/SWMT_timeseries/swmt_shelf_time_pan005_M.nc')

In [ ]:
swmt_shelf_time_anom = swmt_shelf_time_M - swmt_shelf_time_C 

In [ ]:
swmt_shelf_time_anom.plot()

In [ ]:
swmt_shelf_time_anom.to_netcdf('/g/data/g40/kc5856/access_panan_ssp126_SWMT/SWMT_timeseries/swmt_shelf_time_pan005_anom.nc')

## Saving per year via manual method

In [6]:
# Change to your experiment of interest
expt_con = 'panant-005-zstar-ACCESSyr2'
db = '/home/272/kc5856/databases/honours_test.db'
session = cc.database.create_session(db)
freq = '1 monthly'

# Select time period and region
lon_slice = slice(None, None)
lat_slice = slice(None, -59)

In [7]:
# Checking get_var function
start_time = '2001-01-01'
end_time = '2001-12-31'
ds = get_variables(expt_con, freq, start_time, end_time, lon_slice, lat_slice, model = 'mom6')
ds

Exception: Data loading error: var=thetao, freq=1 monthly, start_time=2001-01-01, end_time=2001-12-31, expt=panant-005-zstar-ACCESSyr2: Every dimension requires a corresponding 1D coordinate and index for inferring concatenation order but the coordinate 'rho2_l' has no corresponding index

In [86]:
import json
import pandas as pd
from pathlib import Path

# Paths to catalogs
control_json = Path('/home/272/kc5856/catalog_dir/panant_005_zstar_access_yr2.json')
melt_json = Path('/home/272/kc5856/panantrial/panant_005_zstar_MWonly.json')
combined_json = Path('/home/272/kc5856/pan005_combined.json')
combined_csv = Path('/home/272/kc5856/pan005_combined.csv')

# Load both JSON files
with open(control_json, 'r') as f:
    control = json.load(f)

with open(melt_json, 'r') as f:
    melt = json.load(f)

# Remove 'file:' prefix if present
control_catalog_file = control["catalog_file"].replace("file:", "")
melt_catalog_file = melt["catalog_file"].replace("file:", "")

# Get absolute CSV paths
control_csv = Path(control_catalog_file)
melt_csv = Path(melt_catalog_file)

# Read both CSVs
df_control = pd.read_csv(control_csv)
df_melt = pd.read_csv(melt_csv)

# Add a column to identify source (optional but useful)
df_control["source"] = "control"
df_melt["source"] = "meltwater"

# Add a column to identify source (optional but useful)
df_control["experiment"] = "control"
df_melt["experiment"] = "meltwater"


# Combine them
df_combined = pd.concat([df_control, df_melt], ignore_index=True)

# Save combined CSV
df_combined.to_csv(combined_csv, index=False)

# Create new JSON (copy from one and update catalog_file)
combined_metadata = control.copy()
combined_metadata["catalog_file"] = combined_csv.name  # relative path

# Save combined JSON
with open(combined_json, 'w') as f:
    json.dump(combined_metadata, f, indent=2)

print(f"✅ Combined catalog saved to {combined_json}")


✅ Combined catalog saved to /home/272/kc5856/pan005_combined.json


In [89]:
cat = intake.open_esm_datastore("/home/272/kc5856/pan005_combined.json")
cat.search(source=sources)

NameError: name 'sources' is not defined

In [98]:
# Open the esm datastore catalog
expt_cont = intake.open_esm_datastore('/home/272/kc5856/catalog_dir/panant_005_zstar_access_yr2.json')

# Suppose you want to select all variables for your experiment (frequency, etc.)
# Here is an example to get all variables with frequency 'monthly':
selection = expt_cont.search(frequency='monthly')

# Open dataset(s) as xarray dataset dictionary (or combined dataset)
ds_dict = selection.to_dataset_dict()

# If your search returns just one experiment/dataset:
ds_obj = list(ds_dict.values())[0]  # get the single xarray.Dataset

IndexError: list index out of range

### First doing the control

In [45]:
# Painfully manual don't mind this 1
start_time = '2001-01-01'
end_time = '2001-12-31'

ds = get_variables(expt_con, freq, start_time, end_time, lon_slice, lat_slice, model = 'mom6')
           
salt_transformation = compute_salt_transformation(ds)
heat_transformation = compute_heat_transformation(ds)

salt_transformation_binned, heat_transformation_binned = isopycnal_bins(ds, salt_transformation, heat_transformation)

salt_transformation_binned.to_netcdf('/g/data/g40/kc5856/access_panan_ssp126_SWMT/old_Pan005_SWMT_shelf/binned_salt_trans_pan005_C_01.nc')
heat_transformation_binned.to_netcdf('/g/data/g40/kc5856/access_panan_ssp126_SWMT/old_Pan005_SWMT_shelf/binned_heat_trans_pan005_C_01.nc')

Exception: Data loading error: var=thetao, freq=1mon, start_time=2001-01-01, end_time=2001-12-31, expt=panant_005_zstar_access_yr2: 'key=panant_005_zstar_access_yr2 not found in catalog. You can access the list of valid keys via the .keys() method.'

In [ ]:
# Painfully manual don't mind this 2

start_time = '2002-01-01'
end_time = '2002-12-31'

ds = get_variables(expt_con, freq, start_time, end_time, lon_slice, lat_slice, model = 'mom6')
           
salt_transformation = compute_salt_transformation(ds)
heat_transformation = compute_heat_transformation(ds)

salt_transformation_binned, heat_transformation_binned = isopycnal_bins(ds, salt_transformation, heat_transformation)

salt_transformation_binned.to_netcdf('/g/data/g40/kc5856/access_panan_ssp126_SWMT/old_Pan005_SWMT_shelf/binned_salt_trans_pan005_C_02.nc')
heat_transformation_binned.to_netcdf('/g/data/g40/kc5856/access_panan_ssp126_SWMT/old_Pan005_SWMT_shelf/binned_heat_trans_pan005_C_02.nc')

In [ ]:
# Painfully manual don't mind this 3

start_time = '2003-01-01'
end_time = '2003-12-31'

ds = get_variables(expt_con, freq, start_time, end_time, lon_slice, lat_slice, model = 'mom6')
           
salt_transformation = compute_salt_transformation(ds)
heat_transformation = compute_heat_transformation(ds)

salt_transformation_binned, heat_transformation_binned = isopycnal_bins(ds, salt_transformation, heat_transformation)

salt_transformation_binned.to_netcdf('/g/data/g40/kc5856/access_panan_ssp126_SWMT/old_Pan005_SWMT_shelf/binned_salt_trans_pan005_C_03.nc')
heat_transformation_binned.to_netcdf('/g/data/g40/kc5856/access_panan_ssp126_SWMT/old_Pan005_SWMT_shelf/binned_heat_trans_pan005_C_03.nc')

In [ ]:
# Painfully manual don't mind this 4

start_time = '2004-01-01'
end_time = '2004-12-31'

ds = get_variables(expt_con, freq, start_time, end_time, lon_slice, lat_slice, model = 'mom6')
           
salt_transformation = compute_salt_transformation(ds)
heat_transformation = compute_heat_transformation(ds)

salt_transformation_binned, heat_transformation_binned = isopycnal_bins(ds, salt_transformation, heat_transformation)

salt_transformation_binned.to_netcdf('/g/data/g40/kc5856/access_panan_ssp126_SWMT/old_Pan005_SWMT_shelf/binned_salt_trans_pan005_C_04.nc')
heat_transformation_binned.to_netcdf('/g/data/g40/kc5856/access_panan_ssp126_SWMT/old_Pan005_SWMT_shelf/binned_heat_trans_pan005_C_04.nc')

In [ ]:
# Painfully manual don't mind this 5

start_time = '2005-01-01'
end_time = '2005-12-31'

ds = get_variables(expt_con, freq, start_time, end_time, lon_slice, lat_slice, model = 'mom6')
           
salt_transformation = compute_salt_transformation(ds)
heat_transformation = compute_heat_transformation(ds)

salt_transformation_binned, heat_transformation_binned = isopycnal_bins(ds, salt_transformation, heat_transformation)

salt_transformation_binned.to_netcdf('/g/data/g40/kc5856/access_panan_ssp126_SWMT/old_Pan005_SWMT_shelf/binned_salt_trans_pan005_C_05.nc')
heat_transformation_binned.to_netcdf('/g/data/g40/kc5856/access_panan_ssp126_SWMT/old_Pan005_SWMT_shelf/binned_heat_trans_pan005_C_05.nc')

In [ ]:
# Painfully manual don't mind this 6

start_time = '2006-01-01'
end_time = '2006-12-31'

ds = get_variables(expt_con, freq, start_time, end_time, lon_slice, lat_slice, model = 'mom6')
           
salt_transformation = compute_salt_transformation(ds)
heat_transformation = compute_heat_transformation(ds)

salt_transformation_binned, heat_transformation_binned = isopycnal_bins(ds, salt_transformation, heat_transformation)

salt_transformation_binned.to_netcdf('/g/data/g40/kc5856/access_panan_ssp126_SWMT/old_Pan005_SWMT_shelf/binned_salt_trans_pan005_C_06.nc')
heat_transformation_binned.to_netcdf('/g/data/g40/kc5856/access_panan_ssp126_SWMT/old_Pan005_SWMT_shelf/binned_heat_trans_pan005_C_06.nc')

In [ ]:
# Painfully manual don't mind this 7

start_time = '2007-01-01'
end_time = '2007-12-31'

ds = get_variables(expt_con, freq, start_time, end_time, lon_slice, lat_slice, model = 'mom6')
           
salt_transformation = compute_salt_transformation(ds)
heat_transformation = compute_heat_transformation(ds)

salt_transformation_binned, heat_transformation_binned = isopycnal_bins(ds, salt_transformation, heat_transformation)

salt_transformation_binned.to_netcdf('/g/data/g40/kc5856/access_panan_ssp126_SWMT/old_Pan005_SWMT_shelf/binned_salt_trans_pan005_C_07.nc')
heat_transformation_binned.to_netcdf('/g/data/g40/kc5856/access_panan_ssp126_SWMT/old_Pan005_SWMT_shelf/binned_heat_trans_pan005_C_07.nc')

In [ ]:
# Painfully manual don't mind this 8

start_time = '2008-01-01'
end_time = '2008-12-31'

ds = get_variables(expt_con, freq, start_time, end_time, lon_slice, lat_slice, model = 'mom6')
           
salt_transformation = compute_salt_transformation(ds)
heat_transformation = compute_heat_transformation(ds)

salt_transformation_binned, heat_transformation_binned = isopycnal_bins(ds, salt_transformation, heat_transformation)

salt_transformation_binned.to_netcdf('/g/data/g40/kc5856/access_panan_ssp126_SWMT/old_Pan005_SWMT_shelf/binned_salt_trans_pan005_C_08.nc')
heat_transformation_binned.to_netcdf('/g/data/g40/kc5856/access_panan_ssp126_SWMT/old_Pan005_SWMT_shelf/binned_heat_trans_pan005_C_08.nc')

In [ ]:
# Painfully manual don't mind this 9

start_time = '2009-01-01'
end_time = '2009-12-31'

ds = get_variables(expt_con, freq, start_time, end_time, lon_slice, lat_slice, model = 'mom6')
           
salt_transformation = compute_salt_transformation(ds)
heat_transformation = compute_heat_transformation(ds)

salt_transformation_binned, heat_transformation_binned = isopycnal_bins(ds, salt_transformation, heat_transformation)

salt_transformation_binned.to_netcdf('/g/data/g40/kc5856/access_panan_ssp126_SWMT/old_Pan005_SWMT_shelf/binned_salt_trans_pan005_C_09.nc')
heat_transformation_binned.to_netcdf('/g/data/g40/kc5856/access_panan_ssp126_SWMT/old_Pan005_SWMT_shelf/binned_heat_trans_pan005_C_09.nc')

In [ ]:
# Painfully manual don't mind this 10

start_time = '2010-01-01'
end_time = '2010-12-31'

ds = get_variables(expt_con, freq, start_time, end_time, lon_slice, lat_slice, model = 'mom6')
           
salt_transformation = compute_salt_transformation(ds)
heat_transformation = compute_heat_transformation(ds)

salt_transformation_binned, heat_transformation_binned = isopycnal_bins(ds, salt_transformation, heat_transformation)

salt_transformation_binned.to_netcdf('/g/data/g40/kc5856/access_panan_ssp126_SWMT/old_Pan005_SWMT_shelf/binned_salt_trans_pan005_C_10.nc')
heat_transformation_binned.to_netcdf('/g/data/g40/kc5856/access_panan_ssp126_SWMT/old_Pan005_SWMT_shelf/binned_heat_trans_pan005_C_10.nc')

#### Second doing the perturbation:

In [ ]:
# Change to your experiment of interest
expt_melt = 'panant-005-zstar-ssp126-MW-only'
db = '/home/272/kc5856/databases/honours_test.db'
session = cc.database.create_session(db)
freq = '1 monthly'

# Select time period and region
lon_slice = slice(None, None)
lat_slice = slice(None, -59)

In [ ]:
# Confirm its working
start_time = '2001-01-01'
end_time = '2001-12-31'
ds = get_variables(expt_melt, freq, start_time, end_time, lon_slice, lat_slice, model = 'mom6')
ds

In [ ]:
# Painfully manual don't mind this 1

start_time = '2001-01-01'
end_time = '2001-12-31'

ds = get_variables(expt_melt, freq, start_time, end_time, lon_slice, lat_slice, model = 'mom6')
           
salt_transformation = compute_salt_transformation(ds)
heat_transformation = compute_heat_transformation(ds)

salt_transformation_binned, heat_transformation_binned = isopycnal_bins(ds, salt_transformation, heat_transformation)

salt_transformation_binned.to_netcdf('/g/data/g40/kc5856/access_panan_ssp126_SWMT/old_Pan01_SWMT_shelf/binned_salt_trans_pan01_M_01.nc')
heat_transformation_binned.to_netcdf('/g/data/g40/kc5856/access_panan_ssp126_SWMT/old_Pan01_SWMT_shelf/binned_heat_trans_pan01_M_01.nc')

In [ ]:
# Painfully manual don't mind this 2

start_time = '2002-01-01'
end_time = '2002-12-31'

ds = get_variables(expt_melt, freq, start_time, end_time, lon_slice, lat_slice, model = 'mom6')
           
salt_transformation = compute_salt_transformation(ds)
heat_transformation = compute_heat_transformation(ds)

salt_transformation_binned, heat_transformation_binned = isopycnal_bins(ds, salt_transformation, heat_transformation)

salt_transformation_binned.to_netcdf('/g/data/g40/kc5856/access_panan_ssp126_SWMT/old_Pan01_SWMT_shelf/binned_salt_trans_pan01_M_02.nc')
heat_transformation_binned.to_netcdf('/g/data/g40/kc5856/access_panan_ssp126_SWMT/old_Pan01_SWMT_shelf/binned_heat_trans_pan01_M_02.nc')

In [ ]:
# Painfully manual don't mind this 3

start_time = '2003-01-01'
end_time = '2003-12-31'

ds = get_variables(expt_melt, freq, start_time, end_time, lon_slice, lat_slice, model = 'mom6')
           
salt_transformation = compute_salt_transformation(ds)
heat_transformation = compute_heat_transformation(ds)

salt_transformation_binned, heat_transformation_binned = isopycnal_bins(ds, salt_transformation, heat_transformation)

salt_transformation_binned.to_netcdf('/g/data/g40/kc5856/access_panan_ssp126_SWMT/old_Pan01_SWMT_shelf/binned_salt_trans_pan01_M_03.nc')
heat_transformation_binned.to_netcdf('/g/data/g40/kc5856/access_panan_ssp126_SWMT/old_Pan01_SWMT_shelf/binned_heat_trans_pan01_M_03.nc')

In [ ]:
# Painfully manual don't mind this 4

start_time = '2004-01-01'
end_time = '2004-12-31'

ds = get_variables(expt_melt, freq, start_time, end_time, lon_slice, lat_slice, model = 'mom6')
           
salt_transformation = compute_salt_transformation(ds)
heat_transformation = compute_heat_transformation(ds)

salt_transformation_binned, heat_transformation_binned = isopycnal_bins(ds, salt_transformation, heat_transformation)

salt_transformation_binned.to_netcdf('/g/data/g40/kc5856/access_panan_ssp126_SWMT/old_Pan01_SWMT_shelf/binned_salt_trans_pan01_M_04.nc')
heat_transformation_binned.to_netcdf('/g/data/g40/kc5856/access_panan_ssp126_SWMT/old_Pan01_SWMT_shelf/binned_heat_trans_pan01_M_04.nc')

In [ ]:
# Painfully manual don't mind this 5

start_time = '2005-01-01'
end_time = '2005-12-31'

ds = get_variables(expt_melt, freq, start_time, end_time, lon_slice, lat_slice, model = 'mom6')
           
salt_transformation = compute_salt_transformation(ds)
heat_transformation = compute_heat_transformation(ds)

salt_transformation_binned, heat_transformation_binned = isopycnal_bins(ds, salt_transformation, heat_transformation)

salt_transformation_binned.to_netcdf('/g/data/g40/kc5856/access_panan_ssp126_SWMT/old_Pan01_SWMT_shelf/binned_salt_trans_pan01_M_05.nc')
heat_transformation_binned.to_netcdf('/g/data/g40/kc5856/access_panan_ssp126_SWMT/old_Pan01_SWMT_shelf/binned_heat_trans_pan01_M_05.nc')

In [ ]:
# Painfully manual don't mind this 6

start_time = '2006-01-01'
end_time = '2006-12-31'

ds = get_variables(expt_melt, freq, start_time, end_time, lon_slice, lat_slice, model = 'mom6')
           
salt_transformation = compute_salt_transformation(ds)
heat_transformation = compute_heat_transformation(ds)

salt_transformation_binned, heat_transformation_binned = isopycnal_bins(ds, salt_transformation, heat_transformation)

salt_transformation_binned.to_netcdf('/g/data/g40/kc5856/access_panan_ssp126_SWMT/old_Pan01_SWMT_shelf/binned_salt_trans_pan01_M_06.nc')
heat_transformation_binned.to_netcdf('/g/data/g40/kc5856/access_panan_ssp126_SWMT/old_Pan01_SWMT_shelf/binned_heat_trans_pan01_M_06.nc')

In [ ]:
# Painfully manual don't mind this 7

start_time = '2007-01-01'
end_time = '2007-12-31'

ds = get_variables(expt_melt, freq, start_time, end_time, lon_slice, lat_slice, model = 'mom6')
           
salt_transformation = compute_salt_transformation(ds)
heat_transformation = compute_heat_transformation(ds)

salt_transformation_binned, heat_transformation_binned = isopycnal_bins(ds, salt_transformation, heat_transformation)

salt_transformation_binned.to_netcdf('/g/data/g40/kc5856/access_panan_ssp126_SWMT/old_Pan01_SWMT_shelf/binned_salt_trans_pan01_M_07.nc')
heat_transformation_binned.to_netcdf('/g/data/g40/kc5856/access_panan_ssp126_SWMT/old_Pan01_SWMT_shelf/binned_heat_trans_pan01_M_07.nc')

In [ ]:
# Painfully manual don't mind this 8

start_time = '2008-01-01'
end_time = '2008-12-31'

ds = get_variables(expt_melt, freq, start_time, end_time, lon_slice, lat_slice, model = 'mom6')
           
salt_transformation = compute_salt_transformation(ds)
heat_transformation = compute_heat_transformation(ds)

salt_transformation_binned, heat_transformation_binned = isopycnal_bins(ds, salt_transformation, heat_transformation)

salt_transformation_binned.to_netcdf('/g/data/g40/kc5856/access_panan_ssp126_SWMT/old_Pan01_SWMT_shelf/binned_salt_trans_pan01_M_08.nc')
heat_transformation_binned.to_netcdf('/g/data/g40/kc5856/access_panan_ssp126_SWMT/old_Pan01_SWMT_shelf/binned_heat_trans_pan01_M_08.nc')

In [ ]:
# Painfully manual don't mind this 9

start_time = '2009-01-01'
end_time = '2009-12-31'

ds = get_variables(expt_melt, freq, start_time, end_time, lon_slice, lat_slice, model = 'mom6')
           
salt_transformation = compute_salt_transformation(ds)
heat_transformation = compute_heat_transformation(ds)

salt_transformation_binned, heat_transformation_binned = isopycnal_bins(ds, salt_transformation, heat_transformation)

salt_transformation_binned.to_netcdf('/g/data/g40/kc5856/access_panan_ssp126_SWMT/old_Pan01_SWMT_shelf/binned_salt_trans_pan01_M_09.nc')
heat_transformation_binned.to_netcdf('/g/data/g40/kc5856/access_panan_ssp126_SWMT/old_Pan01_SWMT_shelf/binned_heat_trans_pan01_M_09.nc')

In [ ]:
# Painfully manual don't mind this 10

start_time = '2010-01-01'
end_time = '2010-12-31'

ds = get_variables(expt_melt, freq, start_time, end_time, lon_slice, lat_slice, model = 'mom6')
           
salt_transformation = compute_salt_transformation(ds)
heat_transformation = compute_heat_transformation(ds)

salt_transformation_binned, heat_transformation_binned = isopycnal_bins(ds, salt_transformation, heat_transformation)

salt_transformation_binned.to_netcdf('/g/data/g40/kc5856/access_panan_ssp126_SWMT/old_Pan01_SWMT_shelf/binned_salt_trans_pan01_M_10.nc')
heat_transformation_binned.to_netcdf('/g/data/g40/kc5856/access_panan_ssp126_SWMT/old_Pan01_SWMT_shelf/binned_heat_trans_pan01_M_10.nc')